# Run a training script as a command job

You can use the Python SDK for Azure Machine Learning to submit scripts as command jobs. By using jobs, you can easily keep track of the input parameters and outputs when training a machine learning model.

## Before you start

You'll need the latest version of the  **azure-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.22.4
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: c:\users\alienware\miniconda3\envs\py310\lib\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [2]:
import configparser
import os
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient


# Load environment variables from config.ini file
config = configparser.ConfigParser()

# Use an absolute path to avoid path issues
config_file_path = "G:/My Drive/Ingegneria/Data Science GD/My-Practice/my models/Azure ML/config.ini"
config.read(config_file_path)

# all following IDs to be retrieved, to login correctly
os.environ['AZURE_CLIENT_ID'] = config['azure']['client_id']
os.environ['AZURE_CLIENT_SECRET'] = config['azure']['client_secret']
os.environ['AZURE_TENANT_ID'] = config['azure']['tenant_id']
os.environ['AZURE_SUBSCRIPTION_ID'] = config['azure']['subscription_id']
os.environ['AZURE_STORAGE_KEY'] = config['azure']['storage_key']


credential = DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default")

# Initialize MLClient with the obtained credential
ml_client = MLClient(
    credential=credential,
    subscription_id=os.environ['AZURE_SUBSCRIPTION_ID'],
    resource_group_name="rg-dp100-labs",
    workspace_name="mlw-dp100-labs"
)
# ml_client

## Initiate a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train-model-parameters.py** script that can be found in the **src** folder. It uses the **diabetes.csv** file as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command used to test the script in the terminal is the same as the command in the configuration of the job below. 

In [3]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-parameters.py --training_data diabetes.csv --reg_rate 100",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-script-parameters",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.82 MBs): 100%|#######

Monitor your job at https://ml.azure.com/runs/salmon_peach_530mmkhf33?wsid=/subscriptions/a90ed0cd-b0b9-4e3a-bd85-67272a44de15/resourcegroups/rg-dp100-labs/workspaces/mlw-dp100-labs&tid=8bfc37bf-8e21-4420-841d-49303c72ec1a
